In [14]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [15]:
df = pd.read_csv(Path('./data/20210919.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,14/09/2021 17:02:49,EuroTrip (2004),NaN,NaN,NaN,DOGS,NaN,NaN,NaN,NaN
1,14/09/2021 18:03:42,EuroTrip (2004),"The Adventures of Priscilla, Queen of the Dese...",Bring Me the Head of Alfredo Garcia (1974),The Passenger (1975),DOGS,DYING SMALL TOWN,ART HOUSE (..in the middle of our street),[Default to Trish Pic],WARSAW PACT
2,15/09/2021 10:57:41,"The Adventures of Priscilla, Queen of the Dese...",The Passenger (1975),EuroTrip (2004),Bring Me the Head of Alfredo Garcia (1974),ART HOUSE (..in the middle of our street),DOGS,[Default to Trish Pic],DYING SMALL TOWN,WARSAW PACT
3,15/09/2021 19:40:33,Bring Me the Head of Alfredo Garcia (1974),The Passenger (1975),NaN,NaN,WARSAW PACT,DYING SMALL TOWN,ART HOUSE (..in the middle of our street),NaN,NaN
4,16/09/2021 11:55:01,The Passenger (1975),"The Adventures of Priscilla, Queen of the Dese...",NaN,NaN,DYING SMALL TOWN,DOGS,ART HOUSE (..in the middle of our street),WARSAW PACT,[Default to Trish Pic]


In [16]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [17]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['EuroTrip (2004)',
       'The Adventures of Priscilla, Queen of the Desert (1994)',
       'Bring Me the Head of Alfredo Garcia (1974)',
       'The Passenger (1975)'], dtype=object)

In [18]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [19]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [20]:
submission_dict

{'EuroTrip (2004)': <Candidate('EuroTrip (2004)')>,
 'The Adventures of Priscilla, Queen of the Desert (1994)': <Candidate('The Adventures of Priscilla, Queen of the Desert (1994)')>,
 'Bring Me the Head of Alfredo Garcia (1974)': <Candidate('Bring Me the Head of Alfredo Garcia (1974)')>,
 'The Passenger (1975)': <Candidate('The Passenger (1975)')>}

In [21]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [22]:
ballots

[<Ballot(EuroTrip (2004))>,
 <Ballot(EuroTrip (2004), The Adventures of Priscilla, Queen of the Desert (1994), Bring Me the Head of Alfredo Garcia (1974), The Passenger (1975))>,
 <Ballot(The Adventures of Priscilla, Queen of the Desert (1994), The Passenger (1975), EuroTrip (2004), Bring Me the Head of Alfredo Garcia (1974))>,
 <Ballot(Bring Me the Head of Alfredo Garcia (1974), The Passenger (1975))>,
 <Ballot(The Passenger (1975), The Adventures of Priscilla, Queen of the Desert (1994))>]

In [23]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [24]:
print(election_result)

ROUND 1
Candidate                                                  Votes  Status
-------------------------------------------------------  -------  --------
EuroTrip (2004)                                                2  Hopeful
The Passenger (1975)                                           1  Hopeful
The Adventures of Priscilla, Queen of the Desert (1994)        1  Hopeful
Bring Me the Head of Alfredo Garcia (1974)                     1  Rejected

ROUND 2
Candidate                                                  Votes  Status
-------------------------------------------------------  -------  --------
The Passenger (1975)                                           2  Hopeful
EuroTrip (2004)                                                2  Hopeful
The Adventures of Priscilla, Queen of the Desert (1994)        1  Rejected
Bring Me the Head of Alfredo Garcia (1974)                     0  Rejected

FINAL RESULT
Candidate                                                  Votes  Status
-----

In [25]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
DOGS                                             2  Hopeful
DYING SMALL TOWN                                 1  Hopeful
ART HOUSE (..in the middle of our street)        1  Hopeful
WARSAW PACT                                      1  Rejected
[Default to Trish Pic]                           0  Rejected

ROUND 2
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
DOGS                                             2  Hopeful
DYING SMALL TOWN                                 2  Hopeful
ART HOUSE (..in the middle of our street)        1  Rejected
WARSAW PACT                                      0  Rejected
[Default to Trish Pic]                           0  Rejected

FINAL RESULT
Candidate                                    Votes  Status
-----------------------------------------  -------  --------
DOGS